In [3]:
import pandas as pd

def getting_ra_and_dec_perl(field, chip, x, y):
    import os
    import subprocess
    os.chdir("/Users/sishitan/Documents/MOA/inference_packages/RADEC")

    pipe = subprocess.Popen(["perl", "ccd2sky.pl", field, chip, x, y], stdout=subprocess.PIPE)
    result = str(pipe.stdout.read())


    ra_coor, dec_coor = result.split('RA= ')[-1].split('  Dec= ')
    dec_coor = dec_coor.split("\\n'")[0]
    
    return ra_coor, dec_coor

def load_microlensing_meta_data(meta_data_file_path: str) -> pd.DataFrame:
    """
    Loads a microlensing meta data file into a Pandas data frame.

    :param meta_data_file_path: The path to the original meta data CSV.
    :return: The meta data frame.
    """
    
    column_names = ['field', 'chip', 'nsub', 'ID', 'RA', 'Dec', 'x', 'y', 'ndata', 'ndetect', 'sigma', 'sumsigma',
                    'redchi2_out', 'sepmin', 'ID_dophot', 'type', 'mag', 'mage', 't0', 'tE', 'umin', 'fs', 'fb',
                    't0e', 'tEe', 'tEe1', 'tEe2', 'umine', 'umine1', 'umine2', 'fse', 'fbe', 'chi2', 't0FS', 'tEFS',
                    'uminFS', 'rhoFS', 'fsFS', 'fbFS', 't0eFS', 'tEeFS', 'tEe1FS', 'tEe2FS', 'umineFS', 'umine1FS',
                    'umine2FS', 'rhoeFS', 'rhoe1FS', 'rhoe2FS', 'fseFS', 'fbeFS', 'chi2FS']
    meta_dataframe = pd.read_csv(meta_data_file_path, comment='#', header=None, delim_whitespace=True,
                                  names=column_names)
    return meta_dataframe


class Event():
    """
    Event
    """
    def __init__(self, field= None, band= None, chip= None, subfield= None, event_id= None, 
                 event_name = None):
        self.field = field
        self.band = band
        self.chip = chip
        self.subfield = subfield
        self.event_id = event_id 
        self.event_name = event_name
        
    def use_name_to_define(self):
        no_suffix = self.event_name.split(".")[0]
        field, band, chip, subfield, event_id = no_suffix.split("-")
        self.field = field
        self.band = band
        self.chip = chip
        self.subfield = subfield
        self.event_id = event_id 
        return field, band, chip, subfield, event_id
    
    def getting_t0_from_metadata(self, meta_dataframe):
        lightcurve_meta_data = meta_dataframe[(meta_dataframe['ID'] == int(event_id)) &
                                           (meta_dataframe['field'] == field) &
                                           (meta_dataframe['chip'] == int(chip)) &
                                           (meta_dataframe['nsub'] == int(subfield))]

        t0 = lightcurve_meta_data["t0"]
        return t0    

    def getting_RADec_from_metadata(self, meta_dataframe):
        lightcurve_meta_data = meta_dataframe[(meta_dataframe['ID'] == int(event_id)) &
                                           (meta_dataframe['field'] == field) &
                                           (meta_dataframe['chip'] == int(chip)) &
                                           (meta_dataframe['nsub'] == int(subfield))]

        ra_coor = lightcurve_meta_data["RA"]
        dec_coor = lightcurve_meta_data["Dec"]
        return ra_coor, dec_coor   
    
    def getting_XY_from_metadata(self, meta_dataframe):
        lightcurve_meta_data = meta_dataframe[(meta_dataframe['ID'] == int(event_id)) &
                                           (meta_dataframe['field'] == field) &
                                           (meta_dataframe['chip'] == int(chip)) &
                                           (meta_dataframe['nsub'] == int(subfield))]

        x_coor = lightcurve_meta_data["x"]
        y_coor = lightcurve_meta_data["y"]
        return x_coor, y_coor

In [4]:
field = '3' 
chip = '2' 
x = '1963.5000'
y = '256.5700'
print(getting_ra_and_dec_perl(field, chip, x, y))

('17:51:03.707', '-32:14:09.90')


In [5]:
metadata_path = '/Users/sishitan/Documents/MOA/inference_packages/candlist_RADec.dat.txt'
meta_dataframe = load_microlensing_meta_data(metadata_path)

In [6]:
filename = 'gb9-R-4-5-234724.phot.cor'
event = Event(event_name = filename)

field, band, chip, subfield, event_id = event.use_name_to_define()

In [7]:
t0= event.getting_t0_from_metadata(meta_dataframe)
print("t0: ", t0)

ra_coor, dec_coor = event.getting_RADec_from_metadata(meta_dataframe)
print("ra_coor: ",ra_coor,"dec_coor: ", dec_coor)

x_coor, y_coor = event.getting_XY_from_metadata(meta_dataframe)
print("x_coor: ", x_coor, "y_coor: ", y_coor)

print(getting_ra_and_dec_perl(str(field[-1]), str(chip), str(x), str(y)))

t0:  3372    4650.835864
Name: t0, dtype: float64
ra_coor:  3372    17:58:27.368
Name: RA, dtype: object dec_coor:  3372    -29:32:29.33
Name: Dec, dtype: object
x_coor:  3372    1341.19
Name: x, dtype: float64 y_coor:  3372    2034.86
Name: y, dtype: float64
('17:57:07.784', '-29:26:13.19')
